# Deutsch-Josza Algorithm
We have given a function `f: {0, 1}^n->{0, 1}` that is either balanced (50% 0, 50% 1 output) or constant (alwasy 1 or alway 2). Now, we need to decide if the given function is constant or balanced. 

On a conventional computer we can query the function using the different input bitstrings. As soon as we have seen two different outputs we know that the function is balanced. However, if we have measured k-times the same value we only know that the function is constant with probability `P_k=1-1/2^(k-1)`. If we want to be 100% certain we need to query 50% of all `2^n` bitstrings, i.e., `2^(n-1)` queries.

The Deutsch-Josza algorithm can perform the same task with exactly 1 query using `n+1` qubits.

![Deutsch Josza](deutsch_josza.png)

## Oracles

The oracle `U_f` takes two inputs `|x>` and `|y>` and returns `|x>|y+f(x)>` where `y+f(x)` is understood as addition modulo 2.

In our example we will us a function defined on 2 (q)bits `f: {0,1}^2->{0,1}`. We can define a balanced oracle as

![balanced oracle](oracle_balanced.png)

A constant oracle as

![constant oracle](oracle_constant_1.png)

## Implementation

In [16]:
from typing import List

def check_constant(res: List[bool]) -> bool:
    return all([not _ for _ in res])

In [10]:
from qoqo import Circuit
from qoqo import operations as ops
import qoqo_quest

def deutsch_josza(number_qubits: int, oracle: Circuit) -> Circuit:
    circuit = Circuit()
    circuit += ops.PauliX(number_qubits)
    for q in range(number_qubits):
        circuit += ops.Hadamard(q)
    circuit += ops.Hadamard(number_qubits)
    circuit += oracle
    for q in range(number_qubits):
        circuit += ops.Hadamard(q)
    return circuit

def balanced_oracle(number_qubits: int) -> Circuit:
    oracle = Circuit()
    for c in range(number_qubits):
        oracle += ops.CNOT(control=c, target=number_qubits)
    return oracle

def constant_1_oracle(number_qubits: int) -> Circuit:
    oracle = Circuit()
    oracle += ops.PauliX(number_qubits)
    return oracle

## Simulation
We verify the code by simulating it using qoqo_quest. We only perform one measurement as this is enough to answer the question if the function is constant or balanced.

In [17]:
number_qubits = 2
balanced = Circuit()
balanced += deutsch_josza(
    number_qubits, oracle=balanced_oracle(number_qubits)
)
balanced += ops.DefinitionBit('ro', number_qubits, is_output=True)
for q in range(number_qubits):
    balanced += ops.MeasureQubit(q, 'ro', q)
    
(res, _, _) = qoqo_quest.Backend(number_qubits+1).run_circuit(balanced)
print(res)
print("is constant?", check_constant(res['ro'][0]))

{'ro': [[True, True]]}
False


In [19]:
number_qubits = 2
constant = Circuit()
constant += deutsch_josza(
    number_qubits, oracle=constant_1_oracle(number_qubits)
)
constant += ops.DefinitionBit('ro', number_qubits, is_output=True)
for q in range(number_qubits):
    constant += ops.MeasureQubit(q, 'ro', q)

(res, _, _) = qoqo_quest.Backend(number_qubits+1).run_circuit(constant)
print(res)
print("is constant?", check_constant(res['ro'][0]))

{'ro': [[False, False]]}
is constant? True
